# K-Fold-Crossvalidation with a VGG16

In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib notebook
import sys
#sys.path.append("/home/Deep_Learner/work/network/keras_utils") 
sys.path.append(r"C:\Users\eg38emed\FCD\FCD vs TSC\keras_utils")
import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras import optimizers
from keras import backend
from keras import applications
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D, Dense, Input, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, AveragePooling2D, Concatenate
from image.image_data_generator import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array
from image.directory_iterator import DirectoryIterator
from image.dataframe_iterator import DataFrameIterator
from keras.metrics import categorical_accuracy
from keras.applications.xception import preprocess_input
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint, Callback, LambdaCallback, CSVLogger

#from keras.preprocessing.image import ImageDataGenerator

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import pandas as pd
import imgaug as ia
from imgaug import augmenters as iaa
from pathlib import Path
import cv2
import os
import shutil
from time import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc

from helper import *
auc_roc = as_keras_metric(tf.metrics.auc)
import clr_callback

Using TensorFlow backend.


Sequence


In [ ]:
'''path = '/home/Deep_Learner/work/local/Tiles_RCZ/FCD/'
train_dir = '/home/Deep_Learner/work/local/Tiles_RCZ/Train/a_fcd/'
file = '/home/Deep_Learner/work/local/FCD.csv'
df=pd.read_csv(file)
#print(df.column)
df_fcd = df.drop('path', 1)
#df_fcd = df_fcd.reset_index()
print(fname)
print(df_fcd.head())
print(len(df_fcd))

for j, image in enumerate(df_fcd['id']):
        # get the label for a certain image
        print(j)
        print(image)
        fname = image  
        # source path to image
        src = os.path.join(path, fname)
        print(src)
        # destination path to image
        dst = os.path.join(train_dir, fname)
        print(dst)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)'''

In [2]:
import imgaug as ia
from imgaug import augmenters as iaa
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.5), # vertically flip 20% of all images
        sometimes(iaa.Affine(
            scale={"x": (0.9, 1.6), "y": (0.9, 1.6)}, #>20 will cut part of img
            translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, # >20% will also cut part of img
            rotate=(-10, 10), # 45/-45° -> works good with scale + translate to prevent cuts
            shear=(-5, 5), # shear by -16 to +16 degrees
            mode=ia.ALL 
        )),
        iaa.SomeOf((0, 4), [
                sometimes(iaa.Superpixels(p_replace=(0.3, 0.7), n_segments=(10, 100))), #superpixel-representation --> better basallamina representation 
                iaa.OneOf([
                    iaa.GaussianBlur((0, 0.2)), #small blur effects --> better representation
                    iaa.AverageBlur(k=(1, 3)), # k must be odd
                    iaa.MedianBlur(k=(1, 3)), # 
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)), #cell wall represenation
                iaa.Emboss(alpha=(0, 0.8), strength=(0, 0.5)), #cell wall represenation
                #searching for edges or angles --> blobby mask --> better basallamina representation / nuclei
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.2, 0.4)), #detects edges --> cell wall,..
                    iaa.DirectedEdgeDetect(alpha=(0.2, 0.4), direction=(0.0, 1.0)), #direction will make edges from random directions 
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255), per_channel=0.2), # add gaussian noise to images
             iaa.OneOf([
                    iaa.Dropout((0.05, 0.3), per_channel=0.2), #rnd remove 5% in small pixels
                    iaa.CoarseDropout((0.05, 0.3), size_percent=(0.01, 0.02), per_channel=0.2),# rnd remove 3% in big pixels
                ]),
                iaa.Invert(0.01, per_channel=True), # invert color channels
                iaa.Add((-10, 10), per_channel=0.3), # change brightness of images (by -10 to 10 of original value)
                #iaa.AddToHueAndSaturation((-0.1, 0.1)), # change hue and saturation
                #
                #either change the brightness of the whole image (sometimes per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.9, 1.2), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-1, 0),
                        first=iaa.Multiply((0.9, 1.1), per_channel=True),
                        second=iaa.ContrastNormalization((0.9, 1.1))
                    )
                ]),
                sometimes(iaa.ElasticTransformation(alpha=(0, 0.5), sigma=0.1)), #still not sure: move pixels locally around
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.03))), #still not sure:move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
                     random_order=True
        )
    ],
    random_order=True
)

### k-fold crossvalidation with tiles in RAM

In [ ]:
#doesnt match the unshuffled numbers???
i=0
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    print('[Fold %d/%d]' % (i + 1, kfold))
    i=i+1
    #slice training-data into folds for training/testing variables
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    #get .fit values
    num_train_samples = len(X_train)
    num_val_samples = len(X_valid)  
    train_steps = np.ceil(num_train_samples // train_batch_size)
    val_steps = np.ceil(num_val_samples // val_batch_size)
    
    #get data-generators
    train_generator = datagen.flow(X_train, y_train, batch_size = train_batch_size)
    val_generator = datagen_test.flow(X_valid, y_valid, batch_size = val_batch_size)
    
    #Callbacks & Model
    name_weights = (base_path + "\Fold_" + str(i) + "{}.h5" .format(model_name))
    callbacks_list = get_callbacks(name_weights = name_weights)
    model_VGG = get_model_classif_VGG_base_nottrainable()
 
    #first on small batch_size to establish the basic informations
    history = model_VGG.fit_generator(
        train_generator,
        epochs=5, verbose=1,
        validation_data = val_generator,
        steps_per_epoch=train_steps,
        validation_steps=val_steps,
        callbacks=callbacks_list)

    #second round --> whole model with larger batchsize
    num_train_samples = len(X_train)
    num_val_samples = len(X_valid)  
    train_steps = np.ceil(num_train_samples // train_batch_size_2)
    val_steps = np.ceil(num_val_samples // val_batch_size)
    
    #get data-generators
    train_generator = datagen.flow(X_train, y_train, batch_size = train_batch_size_2)
    val_generator = datagen_test.flow(X_valid, y_valid, batch_size = val_batch_size) 
    model_VGG_bt = get_model_classif_VGG_base_trainable()
    model_VGG_bt.load_weights(name_weights)
        
    history = model_VGG_bt.fit_generator(
        train_generator,
        epochs=50, verbose=1,
        validation_data = val_generator,
        steps_per_epoch=train_steps,
        validation_steps=val_steps,
        callbacks=callbacks_list)
    
    print(model_VGG.evaluate(X_valid, y_valid))

### Create Folder Structure for tiles -> debugging purpose

In [ ]:
# Setup data_frame for all data: Path, ID, Label
tiles_path = r"C:\Users\eg38emed\FCD\Tiles\Train_crossvalidation"

fcd = tiles_path + '\FCD'
tsc = tiles_path + '\TSC'

#fcd folder
df = pd.DataFrame({'path': glob(os.path.join(fcd,'*.png'))})
df['id'] = df.path.map(lambda x: x.split('\\')[7].split('.png')[0]) 
df['label'] = 0

#tsc folder
df_pos = pd.DataFrame({'path': glob(os.path.join(tsc,'*.png'))})
df_pos['id'] = df_pos.path.map(lambda x: x.split('\\')[7].split('.png')[0]) 
df_pos['label'] = 1

#concate
df_train = pd.concat([df, df_pos])

#add images
#df_train['image'] = df_train['path'].map(imread)

print(len(df_train))
print(df_train.head())

In [ ]:
#prepare dummy arrays for stratified-k-fold -> only number of items matters
X = np.asarray(df_train['id'])
y = np.asarray(df_train['label'])
print(len(y))
print(len(X))

In [ ]:
#initialize StratifiedKFold
from sklearn.model_selection import StratifiedKFold
kfold = 10
skf = StratifiedKFold(n_splits=kfold, random_state=101, shuffle = False)

train_batch_size_1 = 64
val_batch_size = 64

#datagenerators
datagen_train = ImageDataGenerator(preprocessing_function=seq.augment_image,
                            rescale=1./255)

datagen_val = ImageDataGenerator(rescale=1./255)

In [ ]:
#Creating a new directory-structure --> flow_from_directory + better debugging
file_path = (r'C:\Users\eg38emed\FCD\kfold')
if not os.path.exists(file_path):
    os.makedirs(file_path)

i=0
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    print('[Fold %d/%d]' % (i + 1, kfold))
    i=i+1
'''    y = to_categorical(y)
    #slice training-data into folds for training/testing variables
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
'''
    base_dir = (file_path + r'\base_dir_fold_' + str(i))
    print(base_dir)
    os.mkdir(base_dir)
    # train_dir
    train_dir = os.path.join(base_dir, 'train_dir')
    print(train_dir)
    os.mkdir(train_dir)

    # val_dir
    val_dir = os.path.join(base_dir, 'val_dir')
    os.mkdir(val_dir)

    # Inside each folder we create seperate folders for each class
    # create new folders inside train_dir
    fcd = os.path.join(train_dir, 'a_fcd')
    os.mkdir(fcd)
    tsc = os.path.join(train_dir, 'b_tsc')
    os.mkdir(tsc)

    # create new folders inside val_dir
    fcd = os.path.join(val_dir, 'a_fcd')
    os.mkdir(fcd)
    tsc = os.path.join(val_dir, 'b_tsc')
    os.mkdir(tsc)

    # check that the folders have been created
    print(os.listdir(base_dir + '\\train_dir'))
    print(os.listdir(base_dir + '\\val_dir'))    

    # Transfer the train images
    for j, image in enumerate(train_index):
        # get the label for a certain image
        print(image)
        path = df_train.iloc[image, 0]
        target = path.split('\\')[6]
        print(path)
        fname = df_train.iloc[image, 1]
        fname = fname + '.png'
        # these must match the folder names
        if target == 'FCD':
            label = 'a_fcd'
        if target == 'TSC':
            label = 'b_tsc'    
        # source path to image
        #src = os.path.join(path, fname)
        # destination path to image
        dst = os.path.join(train_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(path, dst)


    # Transfer the val images
    for j, image in enumerate(test_index):
       # get the label for a certain image
        print('Val:', image)
        path = df_train.iloc[image, 0]
        target = path.split('\\')[6]
        print(path)
        fname = df_train.iloc[image, 1]
        fname = fname + '.png'
        # these must match the folder names
        if target == 'FCD':
            label = 'a_fcd'
        if target == 'TSC':
            label = 'b_tsc'   
        # source path to image
        #rc = os.path.join(path, fname)
        # destination path to image
        dst = os.path.join(val_dir, label, fname)
        # copy the image from the source to the destination
        shutil.copyfile(path, dst)

### k-fold crossvalidation on folders

In [3]:
#VGG-16 with BatchNorm + 2. Denselayer
def get_model_classif_VGG_base_nottrainable():
    base_model_VGG = applications.VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    print('Model loaded.')
    
    for layer in base_model_VGG.layers:
        layer.trainable = False
    print("trainable: only toplayer")
    
    x = base_model_VGG.output
    out = GlobalMaxPooling2D(name='1')(x)
    x = BatchNormalization(name='2')(out)
    x = Dropout(0.5, name='3')(x)
    x = Dense(512, activation='relu',name='4') (x)
    x = BatchNormalization(name='5')(x)
    x = Dropout(0.2, name='6')(x)
    x = Dense(256, activation='relu',name='7') (x)
    x = BatchNormalization(name='8')(x)
    
    predictions = (Dense(2, activation='softmax'))(x)
    model_VGG = Model(inputs=base_model_VGG.input, outputs=predictions)

    adam = optimizers.Adam()
    model_VGG.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=[categorical_accuracy, auc_roc])

    model_VGG.summary()
    return model_VGG

def get_model_classif_VGG_base_trainable():
    base_model_VGG = applications.VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    print('Model loaded.')
    
    for layer in base_model_VGG.layers:
        layer.trainable = True
        print("trainable: whole model")
    
    x = base_model_VGG.output
    out = GlobalMaxPooling2D(name='1')(x)
    x = BatchNormalization(name='2')(out)
    x = Dropout(0.5, name='3')(x)
    x = Dense(512, activation='relu',name='4') (x)
    x = BatchNormalization(name='5')(x)
    x = Dropout(0.5, name='6')(x)
    x = Dense(256, activation='relu',name='7') (x)
    x = BatchNormalization(name='8')(x)
    
    predictions = (Dense(2, activation='softmax'))(x)
    model_VGG = Model(inputs=base_model_VGG.input, outputs=predictions)

    adam = optimizers.Adam()
    model_VGG.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=[categorical_accuracy, auc_roc])

    model_VGG.summary()
    return model_VGG

def get_callbacks_clr(name_weights):
    ReduceLR = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, verbose=1, mode='auto', cooldown=5, min_lr=0.00005)
    earlystopping = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0.001, patience=15, verbose=1, mode='auto')
    checkpoint = ModelCheckpoint(name_weights, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
    tensorboard_callback = TensorBoard(log_dir=base_path + "\\logs\\{}".format(time()), 
            histogram_freq=0, #batch_size=32, 
            write_graph=True, write_grads=False, write_images=True, 
            embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
    clr = clr_callback.CyclicLR(base_lr=0.0001, max_lr=0.001,
                        step_size=num_train_samples/train_batch_size_1/2)
    csv_logger = CSVLogger(filename = base_path + "\\model_history_{}_toplayer.csv".format(model_name), append=True)
    return [clr, earlystopping, ReduceLR, checkpoint, tensorboard_callback, csv_logger]

def get_callbacks_clr_2(name_weights):
    ReduceLR = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, verbose=1, mode='auto', cooldown=5, min_lr=0.00005)
    earlystopping = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0.001, patience=15, verbose=1, mode='auto')
    checkpoint = ModelCheckpoint(name_weights, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
    tensorboard_callback = TensorBoard(log_dir=base_path + "\\logs\\{}".format(time()), 
            histogram_freq=0, #batch_size=32, 
            write_graph=True, write_grads=False, write_images=True, 
            embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
    clr = clr_callback.CyclicLR(base_lr=0.000005, max_lr=0.00005,
                        step_size=num_train_samples/train_batch_size_1/2)    
    csv_logger = CSVLogger(filename = base_path + "\\model_history_{}_ALL.csv".format(model_name), append=True)
    return [earlystopping, ReduceLR, checkpoint, tensorboard_callback,  csv_logger]

In [ ]:
#with CSV_Logger
i = 2
kfold = 10
IMAGE_SIZE = 100
IMAGE_CHANNELS = 3
train_batch_size_1 = 128
val_batch_size = 64

while i <= kfold: 
    i +=1
    
    if i == 11:
        break

    model_name = ('VGG_kfold_{}'.format(i))
    base_path = (r'C:\Users\eg38emed\FCD\kfold_RCZ\base_dir_fold_{}'.format(i)) 
    print('Training Model: {} on Fold: {}'.format(model_name, i))
    train_path = (base_path + '\\train_dir') 
    val_path = (base_path + '\\val_dir')

    datagen_train = ImageDataGenerator(preprocessing_function=seq.augment_image,
                                       rotate_random_zoom_crop=True,
                                       rescale=1./255)

    datagen_val = ImageDataGenerator(rescale=1./255,
                                    rotate_random_zoom_crop=True)

    train_gen_1 = DirectoryIterator(train_path, datagen_train,
                                            #target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                            crop_size = (100, 100),
                                            batch_size=train_batch_size_1,
                                            class_mode='categorical')

    val_gen = DirectoryIterator(val_path,
                                datagen_val,
                                #target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                crop_size = (100, 100),
                                batch_size=val_batch_size,
                                class_mode='categorical')
    
    num_train_samples = train_gen_1.samples
    num_val_samples = val_gen.samples

    #train toplayer with cyclic
    model_VGG = get_model_classif_VGG_base_nottrainable()
    name_weights = (base_path + "\\{}_toplayer.h5".format(model_name))
    callbacks_list = get_callbacks_clr(name_weights = name_weights)
    
    history = model_VGG.fit_generator(train_gen_1, 
                                    steps_per_epoch=len(train_gen_1)*20, 
                                    validation_data=val_gen,
                                    validation_steps=len(val_gen)*20,
                                    epochs=13, verbose=1,
                                    use_multiprocessing = False, workers = 6, 
                                    callbacks=callbacks_list)
    plot_history(history, modelname = ('\\{}_toplayer'.format(model_name)), path=base_path)
    
    #FINETUNE ALL VGG with fixed LR
    model_VGG = get_model_classif_VGG_base_trainable()
    model_VGG.load_weights(base_path + "\\{}_toplayer.h5".format(model_name))
    
    name_weights = (base_path + "\\{}_ALL.h5".format(model_name))
    callbacks_list = get_callbacks_clr_2(name_weights = name_weights)
    
    history = model_VGG.fit_generator(train_gen_1, 
                        steps_per_epoch=len(train_gen_1)*20, 
                        validation_data=val_gen,
                        validation_steps=len(val_gen)*20,
                        epochs=20, verbose=1,
                        use_multiprocessing = False, workers = 6, 
                        callbacks=callbacks_list)
    plot_history(history, modelname = ('\\{}_All'.format(model_name)), path=base_path) 

Training Model: VGG_kfold_3 on Fold: 3
Found 4323 images belonging to 2 classes.
Found 481 images belonging to 2 classes.
Model loaded.
trainable: only toplayer
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128

### Retrain some Models

In [ ]:
#1,2,3,4,6,7,8: >90% with 100x100, trained toplayer (0.5 DO, cyclic lr) -> trained model (0.5 DO, 0.0001 lr)
#retrain later: 9: 82% 5: 84%
#retraining some Models: 10: 65% all / 75% in with trained toplayer
i = 3
IMAGE_SIZE = 100
IMAGE_CHANNELS = 3

model_name = ('VGG_kfold_{}'.format(i))
base_path = (r'C:\Users\eg38emed\FCD\kfold_RCZ\base_dir_fold_{}'.format(i)) 
print('Training Model: {} on Fold: {}'.format(model_name, i))
print(base_path)

train_path = (base_path + '\\train_dir') 
val_path = (base_path + '\\val_dir')

train_batch_size_1 = 128
val_batch_size = 64

datagen_train = ImageDataGenerator(preprocessing_function=seq.augment_image,
                                       rotate_random_zoom_crop=True,
                                       rescale=1./255)

datagen_val = ImageDataGenerator(rescale=1./255,
                                rotate_random_zoom_crop=True)

train_gen_1 = DirectoryIterator(train_path, datagen_train,
                                        #target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                        crop_size = (100, 100),
                                        batch_size=train_batch_size_1,
                                        class_mode='categorical')

val_gen = DirectoryIterator(val_path,
                            datagen_val,
                            #target_size=(IMAGE_SIZE,IMAGE_SIZE),
                            crop_size = (100, 100),
                            batch_size=val_batch_size,
                            class_mode='categorical')

#### TRAIN toplayer clr

In [ ]:
#train toplayer with cyclic
model_VGG = get_model_classif_VGG_base_nottrainable()
name_weights = (base_path + "\\{}_toplayer.h5".format(model_name))
callbacks_list = get_callbacks_clr(name_weights = name_weights)

history = model_VGG.fit_generator(train_gen_1, 
                                steps_per_epoch=len(train_gen_1)*20, 
                                validation_data=val_gen,
                                validation_steps=len(val_gen)*20,
                                epochs=13, verbose=1,
                                use_multiprocessing = False, workers = 6, 
                                callbacks=callbacks_list)
plot_history(history, modelname = ('\\{}_toplayer'.format(model_name)), path=base_path)

#### FINETUNE ALL VGG with clr

In [ ]:
#FINETUNE ALL VGG with fixed LR
model_VGG = get_model_classif_VGG_base_trainable()
model_VGG.load_weights(base_path + "\\{}_toplayer.h5".format(model_name))

name_weights = (base_path + "\\{}_ALL.h5".format(model_name))
callbacks_list = get_callbacks_clr_2(name_weights = name_weights)

history = model_VGG.fit_generator(train_gen_1, 
                    steps_per_epoch=len(train_gen_1)*20, 
                    validation_data=val_gen,
                    validation_steps=len(val_gen)*20,
                    epochs=20, verbose=1,
                    use_multiprocessing = False, workers = 6, 
                    callbacks=callbacks_list)
plot_history(history, modelname = ('\\{}_All'.format(model_name)), path=base_path) 